## Predicting FinTech Bootcamp Graduate Salaries
##### Project 2 - Group C (Andrew, Margee, Rachel, Jinhyeong)
##### Presentation Date: January 15, 2022

#### Data Preparation

In [282]:
# Imports
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [283]:
# Create File Path
filepath = "C:/Users/piercerachel/Desktop/git/Project2_GroupC/"
filename = "monster_india_latest_jobs_free_dataset.json"

# Opening JSON file
f = open(filepath + filename)

# returns JSON object as a dictionary
data = json.load(f)

In [284]:
# Create data list
address_country = []
address_locality = []
address_region = []
company = []
description = []
industry = []
postal_code = []
posted_at = []
salary = []  # the target
salary_type = []
skills = []
title = []
url = []

# iterate through each element in the data list
for each in data:

    # append values to list, given each key
    address_country.append(each["address_country"])
    address_locality.append(each["address_locality"])
    address_region.append(each["address_region"])
    company.append(each["company"])
    description.append(each["description"])
    industry.append(each["industry"])
    postal_code.append(each["postal_code"])
    posted_at.append(each["posted_at"])
    salary.append(each["salary"])
    salary_type.append(each["salary_type"])
    skills.append(each["skills"]) 
    title.append(each["title"])
    url.append(each["url"])

In [285]:
# create df, using string header and list as rows
df = pd.DataFrame({"address_country": address_country,
                   "address_locality": address_locality,
                   "address_region": address_region,
                   "company": company,
                   "description": description,
                   "industry": industry,
                   "postal_code": postal_code,
                   "posted_at": posted_at,
                   "salary": salary,
                   "salary_type": salary_type,
                   "skills": skills,
                   "title": title,
                   "url": url,
                  })

df["description"] = df["description"].str.lower()

In [286]:
# Preview dataframe
df.head()

,address_country,address_locality,address_region,company,description,industry,postal_code,posted_at,salary,salary_type,skills,title,url
0,IN,India,India,Kotak Mahindra Bank Limited,job description : - . client engagement o...,Banking/Accounting/Financial Services,NA,2021-09-16,Not disclosed,P.A.,Service Relationship Manager-POS-BRANCH BANKIN...,Service Relationship Manager-POS-BRANCH BANKIN...,https://www.monsterindia.com/job/service-relat...
1,IN,Noida,Uttar Pradesh,ANI Calls India Private Limited,job description : . axway b2bi or axway gatew...,IT/Computers - Software,NA,2021-09-13,1600000,P.A.,Axway B2B EDI Integration consultant,Axway B2B EDI Integration consultant,https://www.monsterindia.com/job/axway-b2b-edi...
2,IN,Chennai,Tamil Nadu,PayPal,job description : job description fueled by a...,Banking/Accounting/Financial Services,NA,2021-08-18,Not disclosed,P.A.,Data Engineer/MTS 2,"MTS 2, Data Engineer",https://www.monsterindia.com/job/mts-2-data-en...
3,IN,Bengaluru / Bangalore,Karnataka,ANI Calls India Private Limited,job description : at least three years weblog...,IT/Computers - Software,NA,2021-08-26,1800000,P.A.,Senior Oracle Fusion Middleware developer,Senior Oracle Fusion Middleware developer,https://www.monsterindia.com/job/senior-oracle...
4,IN,India,India,ANI Calls India Private Limited,job description : • linux engineer with ksh s...,IT/Computers - Software,NA,2021-09-08,1400000,P.A.,Korn Shell (KSH) - Linux System Engineer,Korn Shell (KSH) - Linux System Engineer,https://www.monsterindia.com/job/korn-shell-ks...


In [287]:
# Clean data, drop unnecessary columns
df.drop(['address_country', 'address_locality', 'address_region', 'postal_code', 'salary_type', 'skills', 'url'], axis=1, inplace=True)

In [288]:
df.head()

,company,description,industry,posted_at,salary,title
0,Kotak Mahindra Bank Limited,job description : - . client engagement o...,Banking/Accounting/Financial Services,2021-09-16,Not disclosed,Service Relationship Manager-POS-BRANCH BANKIN...
1,ANI Calls India Private Limited,job description : . axway b2bi or axway gatew...,IT/Computers - Software,2021-09-13,1600000,Axway B2B EDI Integration consultant
2,PayPal,job description : job description fueled by a...,Banking/Accounting/Financial Services,2021-08-18,Not disclosed,"MTS 2, Data Engineer"
3,ANI Calls India Private Limited,job description : at least three years weblog...,IT/Computers - Software,2021-08-26,1800000,Senior Oracle Fusion Middleware developer
4,ANI Calls India Private Limited,job description : • linux engineer with ksh s...,IT/Computers - Software,2021-09-08,1400000,Korn Shell (KSH) - Linux System Engineer


In [289]:
# Drop any salaries that are Not disclosed
df = df[df["salary"] != "Not disclosed"]
df.head()

,company,description,industry,posted_at,salary,title
1,ANI Calls India Private Limited,job description : . axway b2bi or axway gatew...,IT/Computers - Software,2021-09-13,1600000,Axway B2B EDI Integration consultant
3,ANI Calls India Private Limited,job description : at least three years weblog...,IT/Computers - Software,2021-08-26,1800000,Senior Oracle Fusion Middleware developer
4,ANI Calls India Private Limited,job description : • linux engineer with ksh s...,IT/Computers - Software,2021-09-08,1400000,Korn Shell (KSH) - Linux System Engineer
5,ANI Calls India Private Limited,job description : . oracle ebs apps dba imple...,IT/Computers - Software,2021-09-16,1400000,Oracle Cloud Infrastructure
7,Live Connections,requirement mulesoft developer,Recruitment/Staffing/RPO,2021-08-30,2500000,Mulesoft Developer


In [290]:
# Check if there are any NAs/nulls
df.isnull().sum()

company        0
description    0
industry       0
posted_at      0
salary         0
title          0
dtype: int64

In [291]:
# Creating Keywords for Searching
keywords_sql = "sql|oracle|database|db|datawarehouse|datawarehousing"
keywords_python = "python|scripting"
keywords_api = "api"
keywords_algorithm = "algorithm|algorithmic"
keywords_aws = "aws|amazon web services"
keywords_forecast = "forecast|predict"
keywords_bigdata = "bigdata|big data"

In [292]:
# Searching for keyword skills and updating columns from 0s to 1s as applicable
df["sql"] = np.where(df["description"].str.contains(keywords_sql), 1, 0)
df["python"] = np.where(df["description"].str.contains(keywords_python), 1, 0)
df["api"] = np.where(df["description"].str.contains(keywords_api), 1, 0)
df["algorithm"] = np.where(df["description"].str.contains(keywords_algorithm), 1, 0)
df["aws"] = np.where(df["description"].str.contains(keywords_aws), 1, 0)
df["forecast"] = np.where(df["description"].str.contains(keywords_forecast), 1, 0)
df["bigdata"] = np.where(df["description"].str.contains(keywords_bigdata), 1, 0)

In [293]:
# Show updated dataframe
df.head()

,company,description,industry,posted_at,salary,title,sql,python,api,algorithm,aws,forecast,bigdata
1,ANI Calls India Private Limited,job description : . axway b2bi or axway gatew...,IT/Computers - Software,2021-09-13,1600000,Axway B2B EDI Integration consultant,0,0,0,0,0,0,0
3,ANI Calls India Private Limited,job description : at least three years weblog...,IT/Computers - Software,2021-08-26,1800000,Senior Oracle Fusion Middleware developer,1,0,0,0,0,0,0
4,ANI Calls India Private Limited,job description : • linux engineer with ksh s...,IT/Computers - Software,2021-09-08,1400000,Korn Shell (KSH) - Linux System Engineer,0,1,0,0,0,0,0
5,ANI Calls India Private Limited,job description : . oracle ebs apps dba imple...,IT/Computers - Software,2021-09-16,1400000,Oracle Cloud Infrastructure,1,0,0,0,1,0,0
7,Live Connections,requirement mulesoft developer,Recruitment/Staffing/RPO,2021-08-30,2500000,Mulesoft Developer,0,0,0,0,0,0,0


In [294]:
# Drop all rows where all keywords equal 0
df_updated = df[~(df[['sql','python','api','algorithm','aws','forecast','bigdata']] == 0).all(axis=1)]
df_updated.reset_index(drop=True, inplace=True)
df_updated.head()

,company,description,industry,posted_at,salary,title,sql,python,api,algorithm,aws,forecast,bigdata
0,ANI Calls India Private Limited,job description : at least three years weblog...,IT/Computers - Software,2021-08-26,1800000,Senior Oracle Fusion Middleware developer,1,0,0,0,0,0,0
1,ANI Calls India Private Limited,job description : • linux engineer with ksh s...,IT/Computers - Software,2021-09-08,1400000,Korn Shell (KSH) - Linux System Engineer,0,1,0,0,0,0,0
2,ANI Calls India Private Limited,job description : . oracle ebs apps dba imple...,IT/Computers - Software,2021-09-16,1400000,Oracle Cloud Infrastructure,1,0,0,0,1,0,0
3,ANI Calls India Private Limited,job description : . principal engineer cloud ...,IT/Computers - Software,2021-08-23,1400000,Infrastructure as a Service (IaaS),1,0,0,0,0,0,0
4,ANI Calls India Private Limited,job description : . extensive enterprise cons...,IT/Computers - Software,2021-08-30,1800000,SFDC Architect consultant,0,0,1,0,0,0,0


In [295]:
# Convert salary data type before currency conversion
df_updated['salary'] = df_updated['salary'].astype('int')
df_updated.dtypes

company        object
description    object
industry       object
posted_at      object
salary          int32
title          object
sql             int32
python          int32
api             int32
algorithm       int32
aws             int32
forecast        int32
bigdata         int32
dtype: object

In [296]:
# Convert Indian rupees to US dollars (conversion rate is as of 1/8/2022)
df_updated['salary'] = round(df_updated['salary']/74.28, 2)
df_updated.drop(columns=['company', 'description', 'industry', 'posted_at', 'title'], inplace=True)
df_updated.head()

,salary,sql,python,api,algorithm,aws,forecast,bigdata
0,24232.63,1,0,0,0,0,0,0
1,18847.60,0,1,0,0,0,0,0
2,18847.60,1,0,0,0,1,0,0
3,18847.60,1,0,0,0,0,0,0
4,24232.63,0,0,1,0,0,0,0


In [297]:
# Determine data count
df_updated.shape

(175, 8)

In [298]:
# Review new data set
df_updated.describe()

,salary,sql,python,api,algorithm,aws,forecast,bigdata
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,19180.628114,0.605714,0.262857,0.280000,0.057143,0.137143,0.051429,0.057143
std,7907.483036,0.490099,0.441449,0.450287,0.232781,0.344985,0.221504,0.232781
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16828.220000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,18847.600000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,21540.120000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,46445.880000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [299]:
# Next step is to train/test the data!
# salary = y, dummy columns (SQL, etc) are x, split data, train on first 70% of observations with number regression
# Run model on test, see if it performs
# Regression - Predict salary based on skills learned in class OR Classification - cutoff point for salary
# Optional Chatbot of what skills do you possess = salary range in rupees

### DASHBOARD

#### We created a dashboard for the presentation. The dashboard was saved as an html.

##### First Step: Create Markdowns for Overview Tabs

In [ ]:
markdown_titlepage = """

# Predicting FinTech Bootcamp Graduate Salaries  
  
Presentation Date: January 15, 2022 
    
Prepared by Andrew, Rachel, Margee, and Jinhyeong  
  
![image](./images/XXXX.png)

"""

In [ ]:
markdown_intro = """
# Project Objective  
  
Our project is to research and learn what salaries are possible with skills obtained in this FinTech Bootcamp.  
We will be using the Monster India API which includes salaries in rupees since the US-based API required a  
significant fee to use.  We will use the API to retrieve data regarding job descriptions, skills, and salaries  
to determine the following:  
- What jobs require skills from this bootcamp?  
- What are the potential jobs and salaries for graduates with these skills?  
- Can we predict an accurate salary range based on these skills?  

  
![image](./images/XXXX.png)
"""

In [ ]:
markdown_background1 = """
# What Skills Have We Gained as FinTech Bootcamp Students?
  
As part of the U of MN FinTech Bootcamp, we have obtained the necessary skills to automate  
and improve financial services using cutting-edge technology. Skills gained inclde the following:
  
![image](./images/Skills.png)

"""

In [ ]:
markdown_background2 = """
# Background of Key Skills Used During This Project
  
SQL  
Python  
API  
Algorithm  
AWS  
Forecast  
Big data  
  
  
![image](./XX.png)

"""

In [ ]:
markdown_end = """

# Conclusion  
  
Based on our analysis, FinTech Bootcamp graduates can potentially earn a salary between X and X.
  

  
  
![image](./images/XX.png)
"""

In [ ]:
dataphases = """

# Data Phases  
  
Our process consisted of a data exploration, cleanup, and analysis phases.  
  
>- Exploration: Google seaches, API searches, Monster India API, FinTech Bootcamp Curriculum  
>- Cleanup: Searching through variables in Spyder, selecting key data, dropping unnecessary information, narrowing down jobs
>- Analysis: Looking through the data to find jobs that include at least one key skill
>- Training/Testing:
>- Create Model to Predict Salary based on Skills:
>- Fun ChatBot:
  
"""

##### Second Step: Fix any dashboard issues

In [ ]:
#Some of our plots did not translate well in the dashboard, so we converted them to pngs and used the image
plot_1 ="""
![image](./images/plot_1.png)
"""

##### Third Step: Create the Dashboard

In [ ]:
#Create Dashboard
pn.Tabs(
    ("XX",markdown_titlepage),
    ("XX",markdown_background1),
    ("XX",markdown_background2),
    ("XX",markdown_background3),
    ("XX",markdown_background4),
    ("XX",visualoverview),
    ("XX",plot_1),
    ("XX",plot_2),
    ("XX",plot_3),
    ("XX",plot_4),
    ("XX",plot_5),
    ("XX",plot_6),
    ("XX",plot_7),
    ("XX",markdown_end)
       )

##### Final Step: Save to html for presentation

In [ ]:
#Create an html of the dashboard for presentation purposes once finalized
pn.Tabs(
    ("XX",markdown_titlepage),
    ("XX",markdown_background1),
    ("XX",markdown_background2),
    ("XX",markdown_background3),
    ("XX",markdown_background4),
    ("XX",visualoverview),
    ("XX",plot_1),
    ("XX",plot_2),
    ("XX",plot_3),
    ("XX",plot_4),
    ("XX",plot_5),
    ("XX",plot_6),
    ("XX",plot_7),
    ("XX",markdown_end)
       ).save("Dashboard_Presentation.html", embed=True)